Joshua Goldshteyn
Graph Machine Learning Final Project SimpliText
2/21/2023
Main Runner Notebook

In [2]:
# Main Program to generate simplifications of sentences
# importing all necessary modules
import nltk
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
from nltk.tokenize import sent_tokenize, word_tokenize, wordpunct_tokenize
from nltk.probability import FreqDist
import warnings
import numpy as np
from numpy.linalg import norm
warnings.filterwarnings(action = 'ignore')
from pattern.text.en import singularize, pluralize

import gensim
from gensim.models import Word2Vec

import pickle

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Joshua\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Load Word2Vec and Doc2Vec models

In [3]:
w2v_english = pickle.load(open('./w2v_english.pkl', 'rb'))
d2v_english = pickle.load(open('./d2v_english.pkl', 'rb'))

Generate a Simple English Vocabulary

In [4]:
vocab_set = set()
with open('simple_wikipedia_filtered_abstracts.xml', 'r') as file:
    data = file.read()
    tokenized = wordpunct_tokenize(data)
    fdist = FreqDist(tokenized)
    full_set = list(filter(lambda x: x[1]>=10, fdist.items()))
    vocab_set = set(filter(lambda x:x.isalpha(), map(lambda x:x.lower(),map(lambda x: x[0], full_set))))

In [5]:
print(len(vocab_set))

15631


In [6]:
#Adapted from https://drumcoder.co.uk/blog/2013/dec/23/finding-proper-nouns-nltk/
def findtags(tag_prefix, tagged_text):
    """
    Find tokens matching the specified tag_prefix
    """
    cfd = nltk.ConditionalFreqDist((tag, word) for (word, tag) in tagged_text
                                  if tag.startswith(tag_prefix))
    return dict((tag, cfd[tag].keys()) for tag in cfd.conditions())

In [7]:
def replace_word(index, sentence):
    word = sentence[index]
    synonyms = set()
    for syn in wordnet.synsets(word):
        for i in syn.lemmas():
            synonyms.add(i.name())
            #if i.name() not in vocab_set:

    synonyms = list(synonyms)
    #print(word, "SYNONYMS ARE", synonyms)
    #print(set(synonyms))
    #print(set(synonyms) & vocab_set)
    possible_synonyms = set(synonyms).intersection(vocab_set)
    #print(possible_synonyms)
    if len(possible_synonyms)==0:
        return "|"+word+"|"
    #print(word, "POSSIBLE SYNONYMS ARE", possible_synonyms)

    word_affinities = {}
    for cur_word in possible_synonyms:
        if cur_word in w2v_english.wv.key_to_index and word in w2v_english.wv.key_to_index:
            w2v_score = w2v_english.wv.similarity(cur_word, word)
        else:
            w2v_score = 0.25
        #Doc2Vec is very numerically unstable in this use case, and needs to be averaged to get a good value
        sample_count = 25
        d2v_score = 0
        test_sentence = sentence.copy()
        for x in range(sample_count):
            test_sentence[index] = cur_word
            orig = d2v_english.infer_vector(sentence)
            new = d2v_english.infer_vector(test_sentence)
            d2v_score += np.dot(orig,new)/(norm(orig)*norm(new))
        d2v_score /= sample_count
        word_affinities[cur_word] = (w2v_score + d2v_score)/2
    #print("Word Affinities are: ")

    ordered_affinities = sorted(word_affinities.items(), key=lambda x:x[1], reverse=True)
    #print(ordered_affinities)
    chosen_word = ordered_affinities[0][0]
    is_singular = word == singularize(word)
    if is_singular:
        chosen_word = singularize(chosen_word)
    else:
        chosen_word = pluralize(chosen_word)
    if word == chosen_word:
        return chosen_word
    else:
        return "{"+word+"-->"+chosen_word+"}"


In [8]:
def run_simplify(text_to_simplify):
    sentences = sent_tokenize(text_to_simplify)
    output = ""

    for sentence in sentences:
        current_list = word_tokenize(sentence)
        proper_nouns = set()
        proper_nouns_tagged = findtags('NNP', nltk.pos_tag(current_list)).get('NNP')

        if proper_nouns_tagged != None:
            proper_nouns = set(proper_nouns_tagged)
        #print(proper_nouns)
        for x in range(len(current_list)):
            if current_list[x].isalpha() and current_list[x].lower() not in vocab_set and current_list[x] not in proper_nouns:

                current_list[x] = replace_word(x, current_list)
        output+=" ".join(current_list)
        #print(current_list)
    #print(sentences)
    output = output.replace(" , ", ", ")
    output = output.replace(" .", ". ")
    output = output.replace(" !", "! ")
    output = output.replace(" ?", ".? ")
    output = output.replace(" ; ", "; ")
    output = output.replace(" : ", ": ")
    output = output.replace(" 's", "'s")
    output = output.replace(" ( ", " (")
    output = output.replace(" ) ", ") ")
    output = output.replace(" );", ");")
    output = output.replace(" ).", ").")
    print(output)

In [9]:
GNN_text ="""A Graph neural network (GNN) is a class of artificial neural networks for processing data that can be represented as graphs.
In the more general subject of "Geometric Deep Learning", certain existing neural network architectures can be interpreted as GNNs operating on suitably defined graphs.[4] Convolutional neural networks, in the context of computer vision, can be seen as a GNN applied to graphs structured as grids of pixels. Transformers, in the context of natural language processing, can be seen as GNNs applied to complete graphs whose nodes are words in a sentence.

The key design element of GNNs is the use of pairwise message passing, such that graph nodes iteratively update their representations by exchanging information with their neighbors. Since their inception, several different GNN architectures have been proposed,[1][5][6][7] which implement different flavors of message passing.[4] As of 2022, whether it is possible to define GNN architectures "going beyond" message passing, or if every GNN can be built on message passing over suitably defined graphs, is an open research question.[8]

Relevant application domains for GNNs include social networks,[9] citation networks,[10] molecular biology,[11] chemistry,[12] physics[13] and NP-hard combinatorial optimization problems.[14]

Several open source libraries implementing graph neural networks are available, such as PyTorch Geometric[15] (PyTorch), TensorFlow GNN[16] (TensorFlow), and jraph[17] (Google JAX).

The architecture of a generic GNN implements the following fundamental layers:[4]

Permutation equivariant: a permutation equivariant layer maps a representation of a graph into an updated representation of the same graph. In the literature, permutation equivariant layers are implemented via pairwise message passing between graph nodes.[4][8] Intuitively, in a message passing layer, nodes update their representations by aggregating the messages received from their immediate neighbours. As such, each message passing layer increases the receptive field of the GNN by one hop.
Local pooling: a local pooling layer coarsens the graph via downsampling. Local pooling is used to increase the receptive field of a GNN, in a similar fashion to pooling layers in convolutional neural networks. Examples include k-nearest neighbours pooling, top-k pooling,[18] and self-attention pooling.[19]
Global pooling: a global pooling layer, also known as readout layer, provides fixed-size representation of the whole graph. The global pooling layer must be permutation invariant, such that permutations in the ordering of graph nodes and edges do not alter the final output.[20] Examples include element-wise sum, mean or maximum.
It has been demonstrated that GNNs cannot be more expressive than the Weisfeiler–Lehman Graph Isomorphism Test.[21][22] In practice, this means that there exist different graph structures (e.g., molecules with the same atoms but different bonds) that cannot be distinguished by GNNs. More powerful GNNs operating on higher-dimension geometries such as simplicial complexes can be designed.[23] As of 2022, whether or not future architectures will overcome the message passing primitive is an open research question.[8]
"""

run_simplify(GNN_text)

A Graph neural network (GNN) is a class of artificial neural networks for processing data that can be represented as graphs. In the more general subject of `` Geometric Deep Learning '', certain existing neural network architectures can be {interpreted-->interpret} as GNNs operating on |suitably| defined graphs. [ 4 ] Convolutional neural networks, in the context of computer vision, can be seen as a GNN applied to graphs structured as grids of pixels. Transformers, in the context of natural language processing, can be seen as GNNs applied to complete graphs whose nodes are words in a sentence. The key design element of GNNs is the use of |pairwise| message passing, such that graph nodes |iteratively| |update| their representations by {exchanging-->exchange} information with their |neighbors|. Since their {inception-->origin}, several different GNN architectures have been proposed, [ 1 ] [ 5 ] [ 6 ] [ 7 ] which {implement-->enforce} different flavors of message passing. [ 4 ] As of 2022

In [11]:
ML_text = """Machine learning (ML) is a field of inquiry devoted to understanding and building methods that "learn" – that is, methods that leverage data to improve performance on some set of tasks.[1] It is seen as a part of artificial intelligence.

Machine learning algorithms build a model based on sample data, known as training data, in order to make predictions or decisions without being explicitly programmed to do so.[2] Machine learning algorithms are used in a wide variety of applications, such as in medicine, email filtering, speech recognition, agriculture, and computer vision, where it is difficult or unfeasible to develop conventional algorithms to perform the needed tasks.[3][4]

A subset of machine learning is closely related to computational statistics, which focuses on making predictions using computers, but not all machine learning is statistical learning. The study of mathematical optimization delivers methods, theory and application domains to the field of machine learning. Data mining is a related field of study, focusing on exploratory data analysis through unsupervised learning.[6][7]

Some implementations of machine learning use data and neural networks in a way that mimics the working of a biological brain.[8][9]

In its application across business problems, machine learning is also referred to as predictive analytics.
Learning algorithms work on the basis that strategies, algorithms, and inferences that worked well in the past are likely to continue working well in the future. These inferences can be obvious, such as "since the sun rose every morning for the last 10,000 days, it will probably rise tomorrow morning as well". They can be nuanced, such as "X% of families have geographically separate species with color variants, so there is a Y% chance that undiscovered black swans exist".[10]

Machine learning programs can perform tasks without being explicitly programmed to do so. It involves computers learning from data provided so that they carry out certain tasks. For simple tasks assigned to computers, it is possible to program algorithms telling the machine how to execute all steps required to solve the problem at hand; on the computer's part, no learning is needed. For more advanced tasks, it can be challenging for a human to manually create the needed algorithms. In practice, it can turn out to be more effective to help the machine develop its own algorithm, rather than having human programmers specify every needed step.[11]

The discipline of machine learning employs various approaches to teach computers to accomplish tasks where no fully satisfactory algorithm is available. In cases where vast numbers of potential answers exist, one approach is to label some of the correct answers as valid. This can then be used as training data for the computer to improve the algorithm(s) it uses to determine correct answers. For example, to train a system for the task of digital character recognition, the MNIST dataset of handwritten digits has often been used.
"""

In [12]:
run_simplify(ML_text)

Machine learning (ML) is a field of {inquiry-->research} devoted to understanding and building methods that `` learn '' – that is, methods that {leverage-->purchase} data to improve performance on some set of tasks. [ 1 ] It is seen as a part of artificial intelligence. Machine learning algorithms build a model based on sample data, known as training data, in order to make predictions or decisions without being |explicitly| programmed to do so. [ 2 ] Machine learning algorithms are used in a wide variety of applications, such as in medicine, email {filtering-->filter}, speech recognition, agriculture, and computer vision, where it is difficult or |unfeasible| to develop conventional algorithms to perform the needed tasks. [ 3 ] [ 4 ] A subset of machine learning is closely related to computational statistics, which focuses on making predictions using computers, but not all machine learning is statistical learning. The study of mathematical optimization delivers methods, theory and appl

In [14]:
pagerank_text = """
The importance of a Web page is an inherently sub jective matter, which depends on the
readers interests, knowledge and attitudes. But there is still much that can be said ob jectively
about the relative importance of Web pages. This paper describes PageRank, a method for
rating Web pages ob jectively and mechanically, effectively measuring the human interest and
attention devoted to them.
We compare PageRank to an idealized random Web surfer. We show how to eciently
compute PageRank for large numbers of pages. And, we show how to apply PageRank to search
and to user navigation.
1 Introduction and Motivation
The World Wide Web creates many new challenges for information retrieval. It is very large and
heterogeneous. Current estimates are that there are over 150 million web pages with a doubling
life of less than one year. More importantly, the web pages are extremely diverse, ranging from
"What is Joe having for lunch today?" to journals about information retrieval. In addition to these
ma jor challenges, search engines on the Web must also contend with inexperienced users and pages
engineered to manipulate search engine ranking functions.
However, unlike "
at" document collections, the World Wide Web is hypertext and provides
considerable auxiliary information on top of the text of the web pages, such as link structure and
link text. In this paper, we take advantage of the link structure of the Web to produce a global
\importance" ranking of every web page. This ranking, called PageRank, helps search engines and
users quickly make sense of the vast heterogeneity of the World Wide Web.
"""

In [15]:
run_simplify(pagerank_text)

The importance of a Web page is an |inherently| sub |jective| matter, which depends on the readers interests, knowledge and attitudes. But there is still much that can be said |ob| |jectively| about the relative importance of Web pages. This paper describes PageRank, a method for rating Web pages |ob| |jectively| and {mechanically-->automatically}, |effectively| measuring the human interest and attention devoted to them. We compare PageRank to an |idealized| random Web surfer. We show how to eciently {compute-->calculate} PageRank for large numbers of pages. And, we show how to apply PageRank to search and to user navigation. 1 Introduction and Motivation The World Wide Web creates many new challenges for information |retrieval|. It is very large and |heterogeneous|. Current estimates are that there are over 150 million web pages with a {doubling-->double} life of less than one year. More {importantly-->significantly}, the web pages are extremely diverse, ranging from '' What is Joe h